##### Assignment 5 - Classifiers

## About the dataset
The data set you will be using is the cars_base data set from before


## Part 1: Get the DataSet and clean it

1. Read the cars_base.csv file into a data frame

1. Display the info

1. Clean the data set as per last time.

1. Create a "mpg_grade" feature that is a category based on "mpg". Base the mpg_grade so that 0 is the first quartile, 1 is the second quartile, 2 is the third quartile, and 3 is the fourth quartile.  (You can get the quartile values from the describe method of a data frame.) 

1. Split into train and test sets (70% and 30%)



In [12]:
import pandas as pd

def mpg_grade(x):
    if x <= 17.5:
        return 0
    elif x > 17.5 and x <= 23:
        return 1
    elif x > 23 and x <= 29:
        return 2
    else:
        return 3

data = pd.read_csv("cars_base.csv")
hp_mean = data['horsepower'].mean()

data.fillna(value = hp_mean, inplace=True)

data['mpg_grade'] = data['mpg'].apply(mpg_grade)
print(data.describe())

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size = 0.3, random_state=42)
print(len(train_set), len(test_set))

              mpg   cylinders  displacement  horsepower       weight  \
count  398.000000  398.000000    398.000000  398.000000   398.000000   
mean    23.514573    5.454774    193.425879  104.469388  2970.424623   
std      7.815984    1.701004    104.269838   38.199187   846.841774   
min      9.000000    3.000000     68.000000   46.000000  1613.000000   
25%     17.500000    4.000000    104.250000   76.000000  2223.750000   
50%     23.000000    4.000000    148.500000   95.000000  2803.500000   
75%     29.000000    8.000000    262.000000  125.000000  3608.000000   
max     46.600000    8.000000    455.000000  230.000000  5140.000000   

       acceleration  model year      origin   mpg_grade  
count    398.000000  398.000000  398.000000  398.000000  
mean      15.568090   76.010050    1.572864    1.462312  
std        2.757689    3.697627    0.802055    1.123299  
min        8.000000   70.000000    1.000000    0.000000  
25%       13.825000   73.000000    1.000000    0.000000  
50%

## Part 2 - Building the decision tree
Train a decision tree using 
X - Uses your choice of at least 4 features (Not including mpg, or mpg_grade)
y - Classification is "mpg_grade"


In [25]:
from sklearn.tree import DecisionTreeClassifier
X = train_set[['horsepower', 'acceleration', 'weight', 'cylinders']]
Y = train_set['mpg_grade']

tc = DecisionTreeClassifier()
tc.fit(X, Y)

DecisionTreeClassifier()

## Part 3 - Evaluation  (Confusion Matrix)
Create and display the confusion matrix for the training set.  

In [15]:
from sklearn.metrics import confusion_matrix
y_pred = tc.predict(X)
matrix = confusion_matrix(Y, y_pred)
print(matrix)

[[74  0  0  0]
 [ 0 73  0  0]
 [ 0  0 61  0]
 [ 0  0  0 70]]


### Your comments here:
This looks to be a perfect confusion matrix from the jump. There may be a level of overfitting.

## Part 4 - Evaluation (Other metrics)
Compute Accuracy, Precision, Sensitivity and F1 scores from the training data


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy ", accuracy_score(Y, y_pred))
print("Sensitivity ", recall_score(Y, y_pred, average='weighted'))
print("Precision ", precision_score(Y, y_pred, average='weighted'))
print("F1 ", f1_score(Y, y_pred, average='weighted'))



Accuracy  1.0
Sensitivity  1.0
Precision  1.0
F1  1.0


## Part 5 - Displaying the decision tree
Export the decision tree to "salary.dot".

In [17]:
from sklearn.tree import export_graphviz

export_graphviz(tc, out_file='salary.dot')

### Look at the Dot file and answer the following questions:
1. How many nodes are in the tree?
There is 152 nodes.
1. What is the first split
[74, 73, 61, 70]
1. How many leaf nodes are in the tree?  (They will have a lable that just gives a GINI impurity value.)
There are 77 leaf nodes
1. What would you suggest to prevent overfitting?
Doing a K-fold cross validation

### Bonus - Create a .eps or .png file.

To install graphviz, check out https://www.graphviz.org
You will probably need to compile and install graphviz, though there may be an executable version you can download.  

Once you have the dot file, you can render by command line:

```dot -Tps input.dot > output.eps```

```dot -Tpng input.dot > output.png```

## Part 6 (Base Line) : Cross Validate Using A Decision Tree Classifier 

Do a 5 fold cross validation on the entire data set where

X - Uses your choice of at least 4 features (Not including mpg or mpg_grade)
y - Classification is "mpg_grade"

Compute and display the confusion matrix, Accuracy, and F1 score for each fold



In [20]:
from sklearn.model_selection import KFold
validation_accuracy = []
validation_f1 = []
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_ind, valid_set_ind in fold_and_validate.split(X):
    t_set = X.iloc[train_set_ind]
    t_tar = Y.iloc[train_set_ind]
    
    cv_dt = DecisionTreeClassifier()
    cv_dt.fit(t_set, t_tar)
    
    xval = X.iloc[valid_set_ind]
    y_true = Y.iloc[valid_set_ind]
    cvy_pred = cv_dt.predict(xval)
    accuracy = accuracy_score(y_true, cvy_pred)
    f1 = f1_score(y_true, cvy_pred, average='weighted')
    validation_accuracy.append(accuracy)
    validation_f1.append(f1)
    
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 are: ", validation_f1)

Cross validation accuracies are:  [0.7142857142857143, 0.7678571428571429, 0.6428571428571429, 0.7090909090909091, 0.6]
Cross validation f1 are:  [0.7123015873015872, 0.7509711219013545, 0.6265371533228675, 0.7126143922596251, 0.5950630351884272]


## Part 7 - Train and test SVM
Train a linear SVC on the train set.  Use the same features and target as per the baseline.

1. Use the train set to predict the targets, then compute and display the confusion matrix, Accuracy and F1 score.

1. Use the test set to predict the targets, then compute and display the confusion matrix, Accuracy and F1 score.
  

In [30]:
from sklearn.svm import SVC
sc = SVC(kernel='linear')
sc.fit(X,Y)
y_pred = sc.predict(X)
print(confusion_matrix(Y, y_pred))
print("Accuracy ", accuracy_score(Y, y_pred))
print("f1 ", f1_score(Y, y_pred, average='weighted'))


test_X = test_set[['horsepower', 'acceleration', 'weight', 'cylinders']]
test_Y = test_set['mpg_grade']
sc.fit(test_X, test_Y)
test_y_pred = sc.predict(test_X)
print(confusion_matrix(test_Y, test_y_pred))
print("Accuracy ", accuracy_score(test_Y, test_y_pred))
print("f1 ", f1_score(test_Y, test_y_pred, average='weighted'))


[[62 12  0  0]
 [ 6 54 10  3]
 [ 1 10 29 21]
 [ 0  5  9 56]]
Accuracy  0.7230215827338129
f1  0.7197410777900515
[[28  2  0  0]
 [ 3 16 10  1]
 [ 0  1 23  9]
 [ 0  0  4 23]]
Accuracy  0.75
f1  0.7459877885580463


## Part 8 - Cross validate linear SVM
Do a 5 fold cross validation on a linear SVC model using the same features and target as in the Base Line. Compute and display the confusion matrix, accuracy, and F1 score. 


In [33]:
validation_cm = []
validation_accuracy = []
validation_f1 = []
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_ind, valid_set_ind in fold_and_validate.split(X):
    t_set = X.iloc[train_set_ind]
    t_tar = Y.iloc[train_set_ind]
    
    cv_svc = SVC()
    cv_svc.fit(t_set, t_tar)
    
    xval = X.iloc[valid_set_ind]
    y_true = Y.iloc[valid_set_ind]
    cvy_pred = cv_svc.predict(xval)
    matrix = confusion_matrix(y_true, cvy_pred)
    accuracy = accuracy_score(y_true, cvy_pred)
    f1 = f1_score(y_true, cvy_pred, average='weighted')
    validation_cm.append(matrix)
    validation_accuracy.append(accuracy)
    validation_f1.append(f1)
    
print("Cross validation confusion matrices are: ", validation_cm)
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 are: ", validation_f1)

Cross validation confusion matrices are:  [array([[10,  2,  0,  0],
       [ 2, 11,  3,  4],
       [ 1,  2,  1,  7],
       [ 0,  0,  2, 11]], dtype=int64), array([[10,  7,  0,  0],
       [ 1,  9,  2,  0],
       [ 0,  3,  1,  5],
       [ 0,  0,  2, 16]], dtype=int64), array([[15,  1,  0,  0],
       [ 2,  9,  0,  1],
       [ 0,  8,  3,  3],
       [ 0,  3,  1, 10]], dtype=int64), array([[12,  1,  0,  0],
       [ 3, 13,  1,  2],
       [ 0,  1,  4,  6],
       [ 0,  1,  0, 11]], dtype=int64), array([[14,  2,  0,  0],
       [ 0,  7,  1,  2],
       [ 0,  5,  1, 10],
       [ 0,  2,  1, 10]], dtype=int64)]
Cross validation accuracies are:  [0.5892857142857143, 0.6428571428571429, 0.6607142857142857, 0.7272727272727273, 0.5818181818181818]
Cross validation f1 are:  [0.5649459783913565, 0.6279561452372512, 0.6385281385281385, 0.7140594888981986, 0.5351041940515624]


### Compare the results.
1. Based on Part 1, did the SVC model overfit the data?
1. Compare the results of the test set to the cross validation results.
1. How did the linear SVC model perform compared to the decision tree classifier?

1. No the SVC model did not but the original model did.
2. The test set did a lot better than the cross validation. It had higher f1 and accuracy.
3. The model did not do as well but there was a high chance of overfitting in the decision tree classifier.

## Part 9 - Train and test SVM
Train a rbf SVC on the train set and then compute and display metrics for the train and test sets as in the BaseLine
  

In [34]:
sc = SVC(kernel='rbf')
sc.fit(X,Y)
y_pred = sc.predict(X)
print(confusion_matrix(Y, y_pred))
print("Accuracy ", accuracy_score(Y, y_pred))
print("f1 ", f1_score(Y, y_pred, average='weighted'))


sc.fit(test_X, test_Y)
test_y_pred = sc.predict(test_X)
print(confusion_matrix(test_Y, test_y_pred))
print("Accuracy ", accuracy_score(test_Y, test_y_pred))
print("f1 ", f1_score(test_Y, test_y_pred, average='weighted'))

[[61 13  0  0]
 [ 8 51  6  8]
 [ 1 15 16 29]
 [ 0  6  6 58]]
Accuracy  0.6690647482014388
f1  0.6509558712399626
[[26  4  0  0]
 [ 3 16 11  0]
 [ 0  3 22  8]
 [ 0  0  6 21]]
Accuracy  0.7083333333333334
f1  0.7080879348328182


## Part 10 - Cross validate RBF SVM
Do a 5 fold cross validation on a SVC model with an RBF kernel using the same features and target as in the BaseLine. Compute and display metrics as before.


In [36]:
validation_cm = []
validation_accuracy = []
validation_f1 = []
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_ind, valid_set_ind in fold_and_validate.split(X):
    t_set = X.iloc[train_set_ind]
    t_tar = Y.iloc[train_set_ind]
    
    cv_svc = SVC(kernel='rbf')
    cv_svc.fit(t_set, t_tar)
    
    xval = X.iloc[valid_set_ind]
    y_true = Y.iloc[valid_set_ind]
    cvy_pred = cv_svc.predict(xval)
    matrix = confusion_matrix(y_true, cvy_pred)
    accuracy = accuracy_score(y_true, cvy_pred)
    f1 = f1_score(y_true, cvy_pred, average='weighted')
    validation_cm.append(matrix)
    validation_accuracy.append(accuracy)
    validation_f1.append(f1)
    
print("Cross validation confusion matrices are: ", validation_cm)
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 are: ", validation_f1)

Cross validation confusion matrices are:  [array([[10,  2,  0,  0],
       [ 2, 11,  3,  4],
       [ 1,  2,  1,  7],
       [ 0,  0,  2, 11]], dtype=int64), array([[10,  7,  0,  0],
       [ 1,  9,  2,  0],
       [ 0,  3,  1,  5],
       [ 0,  0,  2, 16]], dtype=int64), array([[15,  1,  0,  0],
       [ 2,  9,  0,  1],
       [ 0,  8,  3,  3],
       [ 0,  3,  1, 10]], dtype=int64), array([[12,  1,  0,  0],
       [ 3, 13,  1,  2],
       [ 0,  1,  4,  6],
       [ 0,  1,  0, 11]], dtype=int64), array([[14,  2,  0,  0],
       [ 0,  7,  1,  2],
       [ 0,  5,  1, 10],
       [ 0,  2,  1, 10]], dtype=int64)]
Cross validation accuracies are:  [0.5892857142857143, 0.6428571428571429, 0.6607142857142857, 0.7272727272727273, 0.5818181818181818]
Cross validation f1 are:  [0.5649459783913565, 0.6279561452372512, 0.6385281385281385, 0.7140594888981986, 0.5351041940515624]


### Your comparison with previous results here:
This model did not do as well as the past models. Even the svm linear model had better data when looking at the whole train_set. When the cross validations are completed they did exactly the same as the SVM linear.

## Part 11 - Train and test SVM
Train a polynomial SVC on the train set and then compute and display metrics for the train and test sets (as in the BaseLine).
  

In [37]:
sc = SVC(kernel='poly')
sc.fit(X,Y)
y_pred = sc.predict(X)
print(confusion_matrix(Y, y_pred))
print("Accuracy ", accuracy_score(Y, y_pred))
print("f1 ", f1_score(Y, y_pred, average='weighted'))


sc.fit(test_X, test_Y)
test_y_pred = sc.predict(test_X)
print(confusion_matrix(test_Y, test_y_pred))
print("Accuracy ", accuracy_score(test_Y, test_y_pred))
print("f1 ", f1_score(test_Y, test_y_pred, average='weighted'))

[[62 12  0  0]
 [ 9 49  6  9]
 [ 1 11 18 31]
 [ 0  5  6 59]]
Accuracy  0.6762589928057554
f1  0.6602527200671403
[[26  4  0  0]
 [ 2 15 12  1]
 [ 0  3 14 16]
 [ 0  0  3 24]]
Accuracy  0.6583333333333333
f1  0.6513857780641136


## Part 12 - Cross validate polynomial SVM
Do a 5 fold cross validation on a SVC model with an Polynomial kernel using the same features and target as in the BaseLine. Compute and display metrics as before.


In [38]:
validation_cm = []
validation_accuracy = []
validation_f1 = []
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_ind, valid_set_ind in fold_and_validate.split(X):
    t_set = X.iloc[train_set_ind]
    t_tar = Y.iloc[train_set_ind]
    
    cv_svc = SVC(kernel='poly')
    cv_svc.fit(t_set, t_tar)
    
    xval = X.iloc[valid_set_ind]
    y_true = Y.iloc[valid_set_ind]
    cvy_pred = cv_svc.predict(xval)
    matrix = confusion_matrix(y_true, cvy_pred)
    accuracy = accuracy_score(y_true, cvy_pred)
    f1 = f1_score(y_true, cvy_pred, average='weighted')
    validation_cm.append(matrix)
    validation_accuracy.append(accuracy)
    validation_f1.append(f1)
    
print("Cross validation confusion matrices are: ", validation_cm)
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 are: ", validation_f1)

Cross validation confusion matrices are:  [array([[10,  2,  0,  0],
       [ 2, 11,  3,  4],
       [ 1,  2,  1,  7],
       [ 0,  0,  2, 11]], dtype=int64), array([[10,  7,  0,  0],
       [ 1,  8,  3,  0],
       [ 0,  2,  2,  5],
       [ 0,  0,  1, 17]], dtype=int64), array([[15,  1,  0,  0],
       [ 2,  9,  0,  1],
       [ 0,  5,  6,  3],
       [ 0,  3,  1, 10]], dtype=int64), array([[12,  1,  0,  0],
       [ 3, 11,  3,  2],
       [ 0,  1,  4,  6],
       [ 0,  1,  0, 11]], dtype=int64), array([[14,  2,  0,  0],
       [ 0,  7,  1,  2],
       [ 0,  2,  4, 10],
       [ 0,  1,  2, 10]], dtype=int64)]
Cross validation accuracies are:  [0.5892857142857143, 0.6607142857142857, 0.7142857142857143, 0.6909090909090909, 0.6363636363636364]
Cross validation f1 are:  [0.5649459783913565, 0.6511347642505277, 0.7097402597402597, 0.6766280314667411, 0.6234683366699176]


### Your comparison with previous results here: This model showed even less promise than the two before. The cross validation scores are all somewhat consistent in the .6 range and neither training set or test set confusion matrix did very well.

## Part 13 - Train and test SVM
Train a sigmoid SVC on the train set and then compute and display metrics for the train and test sets (as in the BaseLine).
  

In [39]:
sc = SVC(kernel='sigmoid')
sc.fit(X,Y)
y_pred = sc.predict(X)
print(confusion_matrix(Y, y_pred))
print("Accuracy ", accuracy_score(Y, y_pred))
print("f1 ", f1_score(Y, y_pred, average='weighted'))


sc.fit(test_X, test_Y)
test_y_pred = sc.predict(test_X)
print(confusion_matrix(test_Y, test_y_pred))
print("Accuracy ", accuracy_score(test_Y, test_y_pred))
print("f1 ", f1_score(test_Y, test_y_pred, average='weighted'))

[[ 8  0  0 66]
 [57  0  0 16]
 [59  0  0  2]
 [70  0  0  0]]
Accuracy  0.02877697841726619
f1  0.015891764200579837
[[ 0  0 30  0]
 [ 1  0 29  0]
 [16  0 17  0]
 [24  0  3  0]]
Accuracy  0.14166666666666666
f1  0.08348214285714287


## Part 14 - Cross validate sigmoid SVM
Do a 5 fold cross validation on a SVC model with a signoid kernel using the same features and target as in the BaseLine. Compute and display metrics as before.



In [40]:
validation_cm = []
validation_accuracy = []
validation_f1 = []
fold_and_validate = KFold(n_splits=5, shuffle=True, random_state=145)
for train_set_ind, valid_set_ind in fold_and_validate.split(X):
    t_set = X.iloc[train_set_ind]
    t_tar = Y.iloc[train_set_ind]
    
    cv_svc = SVC(kernel='sigmoid')
    cv_svc.fit(t_set, t_tar)
    
    xval = X.iloc[valid_set_ind]
    y_true = Y.iloc[valid_set_ind]
    cvy_pred = cv_svc.predict(xval)
    matrix = confusion_matrix(y_true, cvy_pred)
    accuracy = accuracy_score(y_true, cvy_pred)
    f1 = f1_score(y_true, cvy_pred, average='weighted')
    validation_cm.append(matrix)
    validation_accuracy.append(accuracy)
    validation_f1.append(f1)
    
print("Cross validation confusion matrices are: ", validation_cm)
print("Cross validation accuracies are: ", validation_accuracy)
print("Cross validation f1 are: ", validation_f1)

Cross validation confusion matrices are:  [array([[ 2,  0,  0, 10],
       [18,  0,  0,  2],
       [10,  0,  0,  1],
       [13,  0,  0,  0]], dtype=int64), array([[ 7,  0,  0, 10],
       [11,  0,  0,  1],
       [ 9,  0,  0,  0],
       [18,  0,  0,  0]], dtype=int64), array([[ 0,  0,  0, 16],
       [ 8,  0,  0,  4],
       [14,  0,  0,  0],
       [14,  0,  0,  0]], dtype=int64), array([[ 1,  0,  0, 12],
       [15,  0,  0,  4],
       [11,  0,  0,  0],
       [12,  0,  0,  0]], dtype=int64), array([[ 2,  0,  0, 14],
       [ 6,  0,  0,  4],
       [15,  0,  0,  1],
       [13,  0,  0,  0]], dtype=int64)]
Cross validation accuracies are:  [0.03571428571428571, 0.125, 0.0, 0.01818181818181818, 0.03636363636363636]
Cross validation f1 are:  [0.015584415584415584, 0.06854838709677419, 0.0, 0.00909090909090909, 0.022377622377622374]


### Your comparison with previous results here: This had the worst response out of any of the models. It has low accuracies and low f1 scores. The confusion matrix showed very little right.

## Propose an explanation for the results that you found for the SVM with the different kernels.

Your explanation here

## Bonus
1. Use a Stochastic Gradient Descent classifier and compare the performance.
1. Use a Random Forrest classifier and compare the performance. 